In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing
from Clasificador import RegresionLogistica, RegresionLineal
from sklearn.model_selection import train_test_split

## Análisis Previo (CVDs)

In [5]:
heartdata=pd.read_csv('heart_failure_clinical_records_dataset.csv')
print(heartdata.shape)

(299, 13)


In [6]:
heartdata.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [7]:
heartdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [8]:
heartdata.describe(include=['int64'])

,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,0.431438,581.839465,0.418060,38.083612,0.351171,136.625418,0.648829,0.32107,130.260870,0.32107
std,0.496107,970.287881,0.494067,11.834841,0.478136,4.412477,0.478136,0.46767,77.614208,0.46767
min,0.000000,23.000000,0.000000,14.000000,0.000000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,0.000000,116.500000,0.000000,30.000000,0.000000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,0.000000,250.000000,0.000000,38.000000,0.000000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,1.000000,582.000000,1.000000,45.000000,1.000000,140.000000,1.000000,1.00000,203.000000,1.00000
max,1.000000,7861.000000,1.000000,80.000000,1.000000,148.000000,1.000000,1.00000,285.000000,1.00000


In [9]:
data = heartdata.to_numpy()
xdata = data[:, :-1]
ydata = data[:, -1]
xdata = preprocessing.scale(xdata)
reg = LogisticRegression(random_state=0)
scores = cross_val_score(reg, xdata, ydata, cv = 5)
print(scores, np.mean(scores), np.std(scores))

NameError: name 'LogisticRegression' is not defined

Con Regresión Logistica conseguimos un score de **0.77** aprox

## Teniendo en cuenta TTE

Fuentes:
 - https://www.publichealth.columbia.edu/research/population-health-methods/time-event-data-analysis
 - https://www.kaggle.com/andrewmvd/heart-failure-clinical-data/discussion/178372

**El análisis previo está mal, estoy usando el tiempo como una variable regresora cuando debe ser parte de la predicción. Esta base de datos es de la forma TTE (Time-To-Event) por tanto intenta predecir tanto un evento como el tiempo en el que se produce. Para hacer un análisis adecuado a lo visto en la asignatura vamos a omitir la columna de tiempo e intentaremos predecir solamente el evento con las 11 características restantes.** 

**El valor medio de la variable tiempo es de 130 días, con una desviación típica de 78 días. Por tanto si no hacemos caso a dicha columna estamos predeciento la muerte o no del paciente en un intervalo de tiempo entre 2 y 6 meses aproximadamente a partir de las mediciones tomadas.** 

Dichas mediciones se realizan con pruebas clínicas a las cuales el paciente se somete por sospecha suya o del doctor de posible CVD. Por tanto nuestro sistema va a predecir la muerte/(supervivencia*) del paciente tras las mediciones en torno a los 4 meses posteriores a las pruebas.

Otra motivación a tener en cuenta cuando analizemos esta bbdd será indicar que atributos son más decisivos para predecir muerte con el fín de evitarlos.

In [18]:
nreps = 100

In [19]:
def prep_cvd_data(data):
    # Primero barajamos los datos
    np.random.shuffle(data)
    # Guardamos en xdata las variables regresoras
    xdata = data[:, :-2]
    # Guardamos en ydata la variable respuesta (evento)
    ydata = data[:, -1]
    # Guardamos en tdata la variable respuesta (tiempo)
    tdata = data[:, -2]
    # Normalizamos las variables regresoras
    xdata = preprocessing.scale(xdata)
    return xdata, ydata, tdata

### Independientes

In [20]:
# Cargamos los datos a una matriz
data = heartdata.to_numpy()
# Definimos lista que guarde todos los scores
S1 = []
S2 = []
# Repetimos nreps veces para sacar el promedio.
for i in range(nreps):
    #Preprocesamiento de datos
    xdata, ydata, tdata = prep_cvd_data(data)
    
    # Definimos la regresión logística
    reg = RegresionLogistica()
    # Particionamos los datos para validar correctamente
    X_train, X_test, y_train, y_test = train_test_split(xdata, ydata, test_size=0.20, random_state=2)
    # Entrenamos
    reg.fit(X_train, y_train)
    # Sacamos el score
    score = reg.score(X_test, y_test)
    
    # Guardamos resultados de ejecución
    S1.append(score)
    # Definimos la regresión lineal
    lin = RegresionLineal()
    # Particionamos, entrenamos y validamos
    X_train, X_test, y_train, y_test = train_test_split(xdata, tdata, test_size=0.20, random_state=2)
    lin.fit(X_train, y_train)
    score = lin.score(X_test, y_test)
    
    # Guardamos resultados de ejecución
    S2.append(score)
# Imprimimos el error promedio cometido en el evento. Medido en Porcentaje.
print("\nCometemos un error de %.2f" % (100*(1 - np.mean(S1))) + " % prediciendo el evento.\n")
# Imprimimos el error promedio cometido en el tiempo. Medido en EAM.
print("\nCometemos un EAM de %.2f" % (np.mean(S2)) + " prediciendo el tiempo.\n")


Cometemos un error de 26.42 % prediciendo el evento.


Cometemos un EAM de -3.17 prediciendo el tiempo.



### Reg-Lin para predecir tiempo y luego Reg-Log para evento

In [21]:
# Definición de variables
data = heartdata.to_numpy()
S1 = []
S2 = []
# Bucle para sacar promedio.
for i in range(nreps):
    #Preprocesamiento de datos
    xdata, ydata, tdata = prep_cvd_data(data)
    
    #Regresión Lineal para sacar el tiempo
    lin = RegresionLineal()
    
    # Particionado y validación
    X_train, X_test, y_train, y_test = train_test_split(xdata, tdata, test_size=0.20, random_state=2)
    lin.fit(X_train, y_train)
    score = lin.score(X_test, y_test)
    
    S1.append(score)   
    
    tpred = lin.predict(xdata)
    # Incorporamos la predicción a las variables regresoras y normalizamos
    xdata2 = np.column_stack((xdata, tpred))
    xdata2 = preprocessing.scale(xdata2)
    # Comprobamos la eficacia prediciendo el evento, de nuevo particionamos y validamos.
    reg = RegresionLogistica()
    X_train, X_test, y_train, y_test = train_test_split(xdata2, ydata, test_size=0.20, random_state=2)
    reg.fit(X_train, y_train)
    score = reg.score(X_test, y_test)
    
    S2.append(score)
# Imprimimos resultados
print("\nCometemos un EAM de %.2f" % (np.mean(S1)) + " prediciendo el tiempo.\n")
print("\nCometemos un error del %.2f" % (100*(1 - np.mean(S2))) + " % prediciendo el evento una vez hemos incorporado la predicción del tiempo.\n")


Cometemos un EAM de -3.05 prediciendo el tiempo.


Cometemos un error del 26.33 % prediciendo el evento una vez hemos incorporado la predicción del tiempo.



###  Reg-Log para evento y luego Reg-Lin para predecir tiempo

In [22]:
# Definición de variables
data = heartdata.to_numpy()
S1 = []
S2 = []
# Bucle para sacar promedio.
for i in range(nreps):
    #Preprocesamiento de datos
    xdata, ydata, tdata = prep_cvd_data(data)
    
    # Regresión Logística para predecir el evento
    reg = RegresionLogistica()
    
    # Particionado y validación
    X_train, X_test, y_train, y_test = train_test_split(xdata, ydata, test_size=0.20, random_state=2)
    reg.fit(X_train, y_train)
    score = reg.score(X_test, y_test)
    
    S1.append(score)
    
    ypred = reg.predict(xdata)
    # Incorporamos la predicción a las variables regresoras y normalizamos
    xdata2 = np.column_stack((xdata, ypred))
    xdata2 = preprocessing.scale(xdata2)
    # Comprobamos la eficacia prediciendo el tiempo, de nuevo particionamos y validamos.
    lin = RegresionLineal()
    
    X_train, X_test, y_train, y_test = train_test_split(xdata2, tdata, test_size=0.20, random_state=2)
    lin.fit(X_train, y_train)
    score = lin.score(X_test, y_test)
    
    S2.append(score)
# Imprimimos resultados
print("\nCometemos un error de %.2f" % (100*(1 - np.mean(S1))) + " % prediciendo el evento.\n")
print("\nCometemos un EAM de %.2f" % (np.mean(S2)) + " prediciendo el tiempo una vez hemos incorporado la predicción del evento.\n")


Cometemos un error de 26.28 % prediciendo el evento.


Cometemos un EAM de -3.17 prediciendo el tiempo una vez hemos incorporado la predicción del evento.



**En este caso tenemos 2 variables respuesta, una de ellas es binaria y la otra es numérica. Para predecir cada una ellas hemos usado regresión logística y lineal respectivamente. Se nos planteó entonces el dilema de si la predicción del tiempo es independiente de la predicción del evento y por tanto hemos probado 3 variantes distintas. La primera consiste en predecir cada variable respuesta usando sólo las 11 variables regresoras. La segunta consiste en predecir primero el tiempo y posteriormente usar esta predicción como variable regresora para predecir el evento. La tercera es idéntica a la segunda solo que predice primero el evento.**

Los resultados obtenidos son prácticamente idénticos en las tres variantes, lo cual nos indiaca que el tiempo de finalización del estudio es independiente del evento por el cual el estudio termina. Obtenemos un error del 26% aproximadamente y un EAM en el tiempo de unos 2 meses. Aunque son errores "decentes" para unas bbdd de sólo 300 ejemplos, pensamos que estas técnicas de regresión lineal y logística no son lo suficientemente potentes en este caso. Hay técnicas mucho más potentes y especializadas en tratar con bastes de datos TTE (Time- To-Event). Como curiosidad, comentamos que usando algoritmos como random-forest o decission-trees se consigue un score en torno al 93-95% aunque esto escapa del temario de esta asignatura.